# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [ ]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
import numpy as np

In [ ]:
(x_train,y_train), (x_test,y_test) = cifar10.load_data()

In [42]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(2393,)
(2393,)
(798,)
(798,)


In [ ]:
y_train = y_train.reshape(y_train.shape[0])
y_test = y_test.reshape(y_test.shape[0])

In [ ]:
x_train_lt5 = x_train[y_train<5]
y_train_lt5 = y_train[y_train<5]
x_test_lt5 = x_test[y_test<5]
y_test_lt5 = y_test[y_test<5]

x_train_gt5 = x_train[y_train>=5]
y_train_gt5 = y_train[y_train>=5]
x_test_gt5 = x_test[y_test>=5]
y_test_gt5 = y_test[y_test>=5]

### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [6]:
import pandas as pd
y_train_lt5 = pd.get_dummies(y_train_lt5)
y_test_lt5 = pd.get_dummies(y_test_lt5)

y_train_gt5 = pd.get_dummies(y_train_gt5)
y_test_gt5 = pd.get_dummies(y_test_gt5)

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [7]:
from keras import Sequential
from keras.layers import Flatten,BatchNormalization,MaxPooling2D,Dropout, Convolution2D, Dense

In [8]:
model = Sequential()
model.add(BatchNormalization(input_shape = (32,32,3)))
model.add(Convolution2D(32,(3,3), activation = 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.2))
model.add(Convolution2D(64,(3,3), activation = 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.2))

model.add(Flatten())
#model.add(BatchNormalization())
model.add(Dense(128,activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(5,activation = 'softmax'))

In [41]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 32, 32, 3)         12        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 6, 6, 64)         

In [10]:
from keras.optimizers import adam
optimizer = adam(lr = 0.001)
model.compile(optimizer = optimizer, loss = 'categorical_crossentropy',metrics = ['accuracy'])
model.fit(x_train_lt5,y_train_lt5, validation_data=(x_test_lt5,y_test_lt5), epochs = 30, batch_size= 32) 

In [11]:
model.evaluate(x_test_lt5,y_test_lt5)

5000/5000 [==============================] - 2s 475us/step


[0.6043273416519165, 0.8214]

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [12]:
for layers in model.layers:
    print(layers.name)
    if('dense' not in layers.name):
        layers.trainable = False

batch_normalization_1
conv2d_1
max_pooling2d_1
dropout_1
conv2d_2
max_pooling2d_2
dropout_2
flatten_1
dense_1
dropout_3
dense_2


In [13]:
#Module to print colourful statements
from termcolor import colored

#Check which layers have been frozen 
for layer in model.layers:
  print (colored(layer.name, 'blue'))
  print (colored(layer.trainable, 'red'))

batch_normalization_1
False
conv2d_1
False
max_pooling2d_1
False
dropout_1
False
conv2d_2
False
max_pooling2d_2
False
dropout_2
False
flatten_1
False
dense_1
True
dropout_3
False
dense_2
True


### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [14]:
from keras.optimizers import adam
optimizer = adam(lr = 0.001)
model.compile(optimizer = optimizer, loss = 'categorical_crossentropy',metrics = ['accuracy'])
model.fit(x_train_gt5,y_train_gt5,validation_data=(x_test_gt5,y_test_gt5), epochs = 10, batch_size= 32)

Train on 25000 samples, validate on 5000 samples
Epoch 1/10
25000/25000 [==============================] - 28s 1ms/step - loss: 1.4174 - acc: 0.5534 - val_loss: 0.6592 - val_acc: 0.7670
Epoch 2/10
25000/25000 [==============================] - 28s 1ms/step - loss: 0.7093 - acc: 0.7438 - val_loss: 0.5337 - val_acc: 0.8128
Epoch 3/10
25000/25000 [==============================] - 28s 1ms/step - loss: 0.6060 - acc: 0.7829 - val_loss: 0.4767 - val_acc: 0.8332
Epoch 4/10
25000/25000 [==============================] - 28s 1ms/step - loss: 0.5343 - acc: 0.8116 - val_loss: 0.4463 - val_acc: 0.8434
Epoch 5/10
25000/25000 [==============================] - 31s 1ms/step - loss: 0.4764 - acc: 0.8291 - val_loss: 0.4284 - val_acc: 0.8518
Epoch 6/10
25000/25000 [==============================] - 28s 1ms/step - loss: 0.4419 - acc: 0.8402 - val_loss: 0.4156 - val_acc: 0.8584
Epoch 7/10
25000/25000 [==============================] - 28s 1ms/step - loss: 0.4127 - acc: 0.8517 - val_loss: 0.4058 - val_acc:

In [15]:
model.evaluate(x_test_gt5,y_test_gt5)

5000/5000 [==============================] - 2s 460us/step


[0.38876817865371704, 0.8728]

## Sentiment analysis <br> 

The objective of the second problem is to perform Sentiment analysis from the tweets data collected from the users targeted at various mobile devices.
Based on the tweet posted by a user (text), we will classify if the sentiment of the user targeted at a particular mobile device is positive or not.

### 6. Read the dataset (tweets.csv) and drop the NA's while reading the dataset

In [16]:
import pandas as pd
data = pd.read_csv('tweets.csv', encoding = "ISO-8859-1").dropna()

In [17]:
data.shape

(3291, 3)

In [18]:
data.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


### Consider only rows having Positive emotion and Negative emotion and remove other rows from the dataframe.

In [19]:
data = data[(data['is_there_an_emotion_directed_at_a_brand_or_product'] == 'Positive emotion') | (data['is_there_an_emotion_directed_at_a_brand_or_product'] == 'Negative emotion')]

In [20]:
data.shape

(3191, 3)

### 7. Represent text as numerical data using `CountVectorizer` and get the document term frequency matrix

#### Use `vect` as the variable name for initialising CountVectorizer.

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
vect.fit(data['tweet_text'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [24]:
dtm = vect.transform(data.tweet_text)

In [25]:
dtm = dtm.toarray()

### 8. Find number of different words in vocabulary

In [26]:
len(dtm)

3191

#### Tip: To see all available functions for an Object use dir

In [ ]:
dir(cv)

### Find out how many Positive and Negative emotions are there.

Hint: Use value_counts on that column

In [27]:
pd.value_counts(data['is_there_an_emotion_directed_at_a_brand_or_product'])

Positive emotion    2672
Negative emotion     519
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: int64

###  Change the labels for Positive and Negative emotions as 1 and 0 respectively and store in a different column in the same dataframe named 'label'

Hint: use map on that column and give labels

In [28]:
data['label'] = data.is_there_an_emotion_directed_at_a_brand_or_product.map({'Positive emotion':1, 'Negative emotion':0})

### 9. Define the feature set (independent variable or X) to be `text` column and `labels` as target (or dependent variable)  and divide into train and test datasets

In [30]:
from sklearn.model_selection import train_test_split
x = data['tweet_text']
y = data['label']
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=12345)

## 10. **Predicting the sentiment:**


### Use Naive Bayes and Logistic Regression and their accuracy scores for predicting the sentiment of the given text

In [31]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

x_train_dtm = vect.fit_transform(x_train)
x_test_dtm = vect.transform(x_test)

In [33]:
nb = MultinomialNB()
nb.fit(x_train_dtm, y_train)
y_pred_class = nb.predict(x_test_dtm)

print (metrics.accuracy_score(y_test, y_pred_class))

0.8709273182957393


In [34]:
logreg = LogisticRegression()
logreg.fit(x_train_dtm, y_train)
y_pred_class = logreg.predict(x_test_dtm)

print (metrics.accuracy_score(y_test, y_pred_class))

0.8784461152882206


E:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


## 11. Create a function called `tokenize_predict` which can take count vectorizer object as input and prints the accuracy for x (text) and y (labels)

In [35]:
def tokenize_test(vect):
    x_train_dtm = vect.fit_transform(x_train)
    print('Features: ', x_train_dtm.shape[1])
    x_test_dtm = vect.transform(x_test)
    nb = MultinomialNB()
    nb.fit(x_train_dtm, y_train)
    y_pred_class = nb.predict(x_test_dtm)
    print('Accuracy: ', metrics.accuracy_score(y_test, y_pred_class))

### Create a count vectorizer function which includes n_grams = 1,2  and pass it to tokenize_predict function to print the accuracy score

In [36]:
# include 1-grams and 2-grams
vect = CountVectorizer(ngram_range=(1, 2))
tokenize_test(vect)

Features:  24694
Accuracy:  0.8721804511278195


### 12. Create a count vectorizer function with stopwords = 'english'  and pass it to tokenize_predict function to print the accuracy score

In [37]:
vect = CountVectorizer(stop_words='english')
tokenize_test(vect)

Features:  4657
Accuracy:  0.8634085213032582


### 13. Create a count vectorizer function with stopwords = 'english' and max_features =300  and pass it to tokenize_predict function to print the accuracy score

In [38]:
vect = CountVectorizer(stop_words='english', max_features=300)
tokenize_test(vect)

Features:  300
Accuracy:  0.8270676691729323


### 14. Create a count vectorizer function with n_grams = 1,2  and max_features = 15000  and pass it to tokenize_predict function to print the accuracy score

In [39]:
vect = CountVectorizer(ngram_range=(1, 2), max_features=15000)
tokenize_test(vect)

Features:  15000
Accuracy:  0.8721804511278195


### 15. Create a count vectorizer function with n_grams = 1,2  and include terms that appear at least 2 times (min_df = 2)  and pass it to tokenize_predict function to print the accuracy score

In [40]:
vect = CountVectorizer(ngram_range=(1, 2), min_df=2)
tokenize_test(vect)

Features:  7665
Accuracy:  0.8721804511278195
